# Gaussian processes

This notebook contains a short **Gaussian process** (GPs) intro. A GP is defined by a **mean function** and a **covariance function**. We start by comparing different isotropic kernels that can be used as covariance functions. This involves the **squared exponential** (radial basis function), the **absolute exponential** (Ornstein-Uhlenbeck) and the **Matérn kernel**. The properties of the GP heavily depend on the chosen kernel. This is demonstrated hereafter by simulating sample paths from GPs with different covariance functions.

In [ ]:
%matplotlib inline

import sys
sys.path.append('..')

In [ ]:
import matplotlib.pyplot as plt
import torch
import torch.distributions as dist
import gpytorch

from utils.kernels import (
    SquaredExponential,
    AbsoluteExponential
)

In [ ]:
torch.set_default_dtype(torch.float64)

## Covariance matrices

In [ ]:
lower = -10
upper = 10
num_coords = 101

coords = torch.linspace(lower, upper, num_coords)

In [ ]:
sigma = 1 # standard deviation
tau = 1 # lengthscale parameter

kernel_rbf = SquaredExponential(sigma=sigma, tau=tau)
cov_rbf = kernel_rbf(coords, coords)

kernel_ou = AbsoluteExponential(sigma=sigma, tau=tau)
cov_ou = kernel_ou(coords, coords)

In [ ]:
nu = 2.5 # smoothness parameter

kernel_matern = gpytorch.kernels.ScaleKernel(
    gpytorch.kernels.MaternKernel(nu=nu)
)
kernel_matern.outputscale = sigma
kernel_matern.base_kernel.lengthscale = tau

for p in kernel_matern.parameters():
    p.requires_grad = False

cov_matern = kernel_matern(coords, coords).evaluate()

In [ ]:
cov_dict = {
    'RBF': cov_rbf,
    'OU': cov_ou,
    'Matern': cov_matern
}

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(9, 2.5))
imgs = []
for ax, (key, cov) in zip(axes.ravel(), cov_dict.items()):
    img = ax.matshow(cov.numpy(), cmap='viridis', vmin=0)
    ax.set_aspect('equal', adjustable='box')
    ax.set_title(key)
    imgs.append(img)
for ax, img in zip(axes, imgs):
    fig.colorbar(img, ax=ax)
fig.tight_layout()

## Sampling

In [ ]:
num_samples = 10

samples_rbf = dist.MultivariateNormal(
    loc=torch.zeros_like(coords),
    covariance_matrix=cov_rbf + 1e-07*torch.eye(num_coords)
).sample((num_samples,))

samples_ou = dist.MultivariateNormal(
    loc=torch.zeros_like(coords),
    covariance_matrix=cov_ou + 1e-07*torch.eye(num_coords)
).sample((num_samples,))

samples_matern = dist.MultivariateNormal(
    loc=torch.zeros_like(coords),
    covariance_matrix=cov_matern
).sample((num_samples,))

In [ ]:
samples_dict = {
    'RBF': samples_rbf,
    'OU': samples_ou,
    'Matern': samples_matern
}

fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(6, 12))
for ax, (key, samples) in zip(axes.ravel(), samples_dict.items()):    
    ax.plot(coords.numpy(), samples.T.numpy(), alpha=0.7)
    ax.set(xlabel='coordinate', ylabel='value')
    ax.set_xlim((coords[0], coords[-1]))
    ax.grid(visible=True, which='both', color='lightgray', linestyle='-')
    ax.set_axisbelow(True)
    ax.set_title(key)
fig.tight_layout()